### Reproduction of Inception Time algo applied to Fofu dataset from paper by Verket

Run cells to set github automatically

In [1]:
!git config --global user.email "a.kiselev@student.tudelft.nl"
!git config --global user.name "Alexander Kiselev"

Calls to install on the needed packages as the environment gets wiped every time

In [2]:
!pip install llvmlite==0.38.0 --ignore-installed
!pip install tsai -U >> /dev/null
!pip install --upgrade sktime==0.9.0
!pip3 install --upgrade numpy==1.20
!pip install wandb -U
!pip install optuna -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 34.5 MB 9.7 MB/s eta 0:00:011
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.52.0 requires llvmlite<0.36,>=0.35.0, but you have llvmlite 0.38.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 15.4 MB 12.8 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.3
    Uninstalling numpy-1.19.3:
      Successfully uninstalled numpy-1.19.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.9.0 requires numpy<=1.19.3, but y

In [4]:
import numpy as np
import torch
from tsai.all import *
from scipy import io
import wandb
from fastai.callback.wandb import *
from getdata import get_fofu_data
my_setup(wandb)

os             : Linux-5.4.0-89-generic-x86_64-with-glibc2.10
python         : 3.8.12
tsai           : 0.2.25
fastai         : 2.5.3
fastcore       : 1.3.27
wandb          : 0.12.10
torch          : 1.10.0a0+0aef44c
n_cpus         : 8
device         : cuda (Quadro RTX 4000)


In [5]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Define the sweep

In [6]:
sweep_config = {
    "name" : "Sweep on LR, Wd, and Dropout using Standardized data with accuracy as metric",
    "project" : "HOClass",
    "description" : "A sweep using Bayes search, adjusted range on Wd, decreased lower bound on Conv_dropout factor, added data standardization using Verskeer method, changed bottleneck to always off, switched residual to always off, switched LR to use a range",
    "method" : "bayes",
    "metric": {
        "name": "accuracy",
        "goal": "maximize",
    },
    "early_terminate" : {
        "type" : "hyperband",
        "min_iter" : 7500,
        "eta" : 2,
    },
  
    "parameters" : {
        "epochs" : {
            "value" : 75
        },
        "lr" : {
            "min": 0.001,
            "max" : 0.025,
        },
        "batch_size" : {
            "value" : 64
        },
        "nf" : {
            "value" : 24,
        },
        "ks" : {
            "value" : 64
        },
        "bottleneck" : {
            "value" : False
        },
        "bottleneck_size"  : {
            "value" : 0
        },
        "depth"  : {
            "value" :  6
        },
        "residual"  : {
            "value" : False
        },
        "valid_pct"  : {
            "value" : 0.2
        },
        "wd" : {
            "distribution" : "log_uniform",
            "min" : -4.5,
            "max" : -0.5,
        },
        "conv_dropout" : {
            "min" : 0.1,
            "max" : 0.75,
        },
        "variables" : {
            "value" : ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt']
        },
        "bn" : {
            "values" : [True, False]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="HOClass")

Create sweep with ID: 7g0f0hky
Sweep URL: https://wandb.ai/sashalikesplanes/HOClass/sweeps/7g0f0hky


Current sweep id : 7g0f0hky

In [6]:
sweep_id = "7g0f0hky"

In [7]:
def train():
    with wandb.init(project="HOClass") as run:
        config = wandb.config
        # Get the data sets
        print(config)
        X, Y, splits = get_fofu_data(valid_pct=config["valid_pct"])
        print(Y)
        tfms = [None, Categorize()]
        dsets = TSDatasets(X, Y, tfms=tfms, splits=splits, inplace=True)
        
        # Get data loaders
        batch_tfms = None #[TSStandardize(by_var=config["standardize_by_var"], by_sample=config["standardize_by_sample"])]
        dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config["batch_size"], batch_tfms=batch_tfms, shuffle=True, num_workers=8)
        
        # Get model
        cbs = [SaveModelCallback(), WandbCallback(log_preds=True, log_model=True, dataset_name='valid runs random, standardized')]
        model = InceptionTimePlus(dls.vars, dls.c, nf=config["nf"], bn=config["bn"], ks=config["ks"], bottleneck=config["bottleneck"],
                                  conv_dropout=config["conv_dropout"], depth=config["depth"])
        # Get learner
        learner = Learner(dls, model, opt_func=Adam, metrics=accuracy, wd=config["wd"], cbs=cbs)
        # Execute fit one cycle training
        learner.fit_one_cycle(config["epochs"], lr_max=config["lr"])

In [8]:
wandb.agent(sweep_id, project="HOClass", function=train)

wandb: Agent Starting Run: 01auwme0 with config:
wandb: 	batch_size: 64
wandb: 	bn: False
wandb: 	bottleneck: False
wandb: 	bottleneck_size: 0
wandb: 	conv_dropout: 0.239707262307738
wandb: 	depth: 6
wandb: 	epochs: 75
wandb: 	ks: 64
wandb: 	lr: 0.010045000833772529
wandb: 	nf: 24
wandb: 	residual: False
wandb: 	valid_pct: 0.2
wandb: 	variables: ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt']
wandb: 	wd: 0.044800905585340854
wandb: Currently logged in as: sashalikesplanes (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 64, 'bn': False, 'bottleneck': False, 'bottleneck_size': 0, 'conv_dropout': 0.239707262307738, 'depth': 6, 'epochs': 75, 'ks': 64, 'lr': 0.010045000833772529, 'nf': 24, 'residual': False, 'valid_pct': 0.2, 'variables': ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt'], 'wd': 0.044800905585340854}


Run 01auwme0 errored: ValueError('cannot reshape array of size 540000 into shape (12000,54)')
wandb: ERROR Run 01auwme0 errored: ValueError('cannot reshape array of size 540000 into shape (12000,54)')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4q61s6wy with config:
wandb: 	batch_size: 64
wandb: 	bn: True
wandb: 	bottleneck: False
wandb: 	bottleneck_size: 0
wandb: 	conv_dropout: 0.17994242148902723
wandb: 	depth: 6
wandb: 	epochs: 75
wandb: 	ks: 64
wandb: 	lr: 0.008400061183635739
wandb: 	nf: 24
wandb: 	residual: False
wandb: 	valid_pct: 0.2
wandb: 	variables: ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt']
wandb: 	wd: 0.31441165420637424
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 64, 'bn': True, 'bottleneck': False, 'bottleneck_size': 0, 'conv_dropout': 0.17994242148902723, 'depth': 6, 'epochs': 75, 'ks': 64, 'lr': 0.008400061183635739, 'nf': 24, 'residual': False, 'valid_pct': 0.2, 'variables': ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt'], 'wd': 0.31441165420637424}


Run 4q61s6wy errored: ValueError('cannot reshape array of size 540000 into shape (12000,54)')
wandb: ERROR Run 4q61s6wy errored: ValueError('cannot reshape array of size 540000 into shape (12000,54)')
wandb: Agent Starting Run: hbjfyg3t with config:
wandb: 	batch_size: 64
wandb: 	bn: False
wandb: 	bottleneck: False
wandb: 	bottleneck_size: 0
wandb: 	conv_dropout: 0.3066140871364286
wandb: 	depth: 6
wandb: 	epochs: 75
wandb: 	ks: 64
wandb: 	lr: 0.003621801131286963
wandb: 	nf: 24
wandb: 	residual: False
wandb: 	valid_pct: 0.2
wandb: 	variables: ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt']
wandb: 	wd: 0.01141345671609547
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 64, 'bn': False, 'bottleneck': False, 'bottleneck_size': 0, 'conv_dropout': 0.3066140871364286, 'depth': 6, 'epochs': 75, 'ks': 64, 'lr': 0.003621801131286963, 'nf': 24, 'residual': False, 'valid_pct': 0.2, 'variables': ['e', 'u', 'x', 'dedt', 'dudt', 'dxdt'], 'wd': 0.01141345671609547}


Run hbjfyg3t errored: ValueError('cannot reshape array of size 540000 into shape (12000,54)')
wandb: ERROR Run hbjfyg3t errored: ValueError('cannot reshape array of size 540000 into shape (12000,54)')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [30]:
config = AttrDict (
    batch_size = 64,
    nf = 24, # number of convolutional filters
    ks = 64, # largest kernel size of convolution filter
    bottleneck = False, # use bottleneck
    bottleneck_size = 0,
    depth = 6, # number of inception models, best 3
    residual = True, # best False
    opt_func = Adam,
    valid_pct = 0.2,
    wd = 5e-6, # weight decay
    lr = 1e-3,
    n_epochs = 50,
    variables = ['e', 'u', 'x' 'dedt', 'dudt', 'dxdt'],
    conv_dropout = 0.1,
    standardize_by_var = True,
    standardize_by_sample = True,
    )


with wandb.init(project="HOClass", config=config, group="InceptionTimePlus", name="All vars, dropout=0.1", save_code=True,):

    
   

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> Expected an input of type in 
  - <class 'numpy.float64'>
  - <class 'fastai.torch_core.TensorCategory'>
 but got <class 'numpy.ndarray'>


epoch,train_loss,valid_loss,accuracy,time
0,0.693357,0.679048,0.689326,00:33
1,0.643246,0.697719,0.680731,00:30
2,0.632047,0.630845,0.714565,00:31
3,0.605246,0.615372,0.721675,00:31
4,0.583077,0.607768,0.724879,00:31
5,0.574802,0.621701,0.725270,00:31
6,0.561086,0.559472,0.748398,00:31
7,0.546220,0.546934,0.758400,00:31
8,0.527859,0.567852,0.748789,00:32
9,0.518102,0.535232,0.766604,00:32


Better model found at epoch 0 with valid_loss value: 0.6790475249290466.
Better model found at epoch 2 with valid_loss value: 0.6308445334434509.
Better model found at epoch 3 with valid_loss value: 0.6153716444969177.
Better model found at epoch 4 with valid_loss value: 0.6077684760093689.
Better model found at epoch 6 with valid_loss value: 0.5594715476036072.
Better model found at epoch 7 with valid_loss value: 0.5469342470169067.
Better model found at epoch 9 with valid_loss value: 0.5352318286895752.
Better model found at epoch 10 with valid_loss value: 0.5163002014160156.
Better model found at epoch 12 with valid_loss value: 0.4812472462654114.
Better model found at epoch 15 with valid_loss value: 0.47725504636764526.
Better model found at epoch 16 with valid_loss value: 0.4745529592037201.
Better model found at epoch 19 with valid_loss value: 0.467233270406723.


accuracy,▁▁▃▃▃▅▅▅▆▆▇▆▇▇▇█▇█▇██▇▇▇▇▇██▇▇▇▇▇▇▇▇▇▇▇▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
mom_0,██▇▆▅▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
raw_loss,█▇▆▇▅▆▇▇▅▇▅▅▅▄▄▄▄▃▃▄▃▄▄▃▃▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
valid_loss,▅▅▄▄▄▃▂▃▂▂▁▂▁▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▅▆▆▆▇▇▇██▇██
wd_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.79395
